Implements PrivateCompare (algorithm 3) in https://eprint.iacr.org/2018/442.pdf.

In [1]:
from tinysmpc import VirtualMachine, PrivateScalar, SharedScalar, Share
from random import random, randint, shuffle

ImportError: cannot import name 'Share' from 'tinysmpc' (/Users/kennysong/GDrive/Projects/tinysmpc/tinysmpc/__init__.py)

In [ ]:
p = 67  # Smaller prime field size to encode bit values
l = 64  # Number of bits of the integers we're using

# TODO: make this work for negative integers?

def get_bits(n, l=64):
    '''Returns the (reverse) binary representation of n as an l-sized list.'''
    bin_str = bin(n).replace('0b', '')
    if len(bin_str) < l:
        bin_str = '0' * (l - len(bin_str)) + bin_str
    return list(map(int, reversed(bin_str)))  # This only works if the bits are reversed, but I don't know why!

def get_bits_ps(n, l=64):
    '''Returns the (reverse) binary representation of PrivateScalar n as an l-sized list.'''
    bin_list = get_bits(n.value)
    return [PrivateScalar(b, n.owner) for b in bin_list]

def randlist(l=64, p=67):
    '''Returns a list of l random integers in [1, p-1].'''
    return [randint(1, p-1) for _ in range(l)]

def fixed_shuffle():
    '''Returns a deterministic shuffle function that always permutes a list in the same way.'''
    seed = random()
    def _fixed_shuffle(x): shuffle(x, lambda: seed)
    return _fixed_shuffle

In [ ]:
def shared_compare(x, r, alice, bob, β=None):
    '''Allows Bob to compute x > r, where x is a PrivateScalar on Alice, and r is public.'''
    # Decompose x into its bit representation, and share each bit independently
    xb = get_bits_ps(x)
    xb_sh = [b.share([bob], Q=p) for b in xb]
    
    # Decompose r into its bit representation, but keep it public
    rb = get_bits(r)
    
    # Common randomness (public)
    β = randint(0, 1) if β is None else β
    s = randlist()
    u = randlist()
    π = fixed_shuffle()

    # Line 1
    t = (r + 1) % 2**l
    tb = get_bits(t)
    
    # Line 2
    w_c = {alice: {'w': [None] * l, 'c': [None] * l}, 
           bob:   {'w': [None] * l, 'c': [None] * l}}
    for j, machine in enumerate([alice, bob]):  
        w, c = w_c[machine]['w'], w_c[machine]['c']
        
        # Line 3
        for i in range(l-1, -1, -1):
            sh = xb_sh[i].share_of[machine]
            
            # Line 4
            if β == 0:
                w[i] = sh + j*rb[i] - 2*rb[i]*sh
                c[i] = j*rb[i] - sh + j + sum(w[i+1:])

            # Line 7
            elif (β == 1) and (r != 2**l - 1):
                w[i] = sh + j*tb[i] - 2*tb[i]*sh
                c[i] = -1*j*tb[i] + sh + j + sum(w[i+1:])

            # Line 10
            else:  
                if i != 1:  c_val = ((1 - j)*(u[i] + 1) - j*u[i]) % p
                else: c_val = ((-1)**j * u[i]) % p
                c[i] = Share(c_val, machine, Q=p)

    # Line 14
    d_alice = [s[i] * w_c[alice]['c'][i] for i in range(l)]
    d_bob = [s[i] * w_c[bob]['c'][i] for i in range(l)]
    π(d_alice); π(d_bob)
    d_shared = [SharedScalar([d_a, d_b], Q=p) for d_a, d_b in zip(d_alice, d_bob)]
    
    # Line 15
    d = [d_sh.reconstruct(bob) for d_sh in d_shared]
    β_prime = any(ps.value == 0 for ps in d)  # Technically, ps == 0 should take place on Bob's machine and return a PrivateScalar
    
    # Return x > r
    return PrivateScalar(β ^ β_prime, bob)

In [ ]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
x = PrivateScalar(100, alice)
r = 0

assert shared_compare(x, r, alice, bob).value == 1

In [ ]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
x = PrivateScalar(100, alice)
r = 99

assert shared_compare(x, r, alice, bob).value == 1

In [ ]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
x = PrivateScalar(100, alice)
r = 100

assert shared_compare(x, r, alice, bob).value == 0

In [ ]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
x = PrivateScalar(100, alice)
r = 101

assert shared_compare(x, r, alice, bob).value == 0

In [ ]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
x = PrivateScalar(100, alice)
r = 1001

assert shared_compare(x, r, alice, bob).value == 0

In [ ]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
x = PrivateScalar(1001, alice)
r = 0

assert shared_compare(x, r, alice, bob).value == 1

In [ ]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
x = PrivateScalar(1001, alice)
r = 1001

assert shared_compare(x, r, alice, bob).value == 0

In [ ]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
x = PrivateScalar(1001, alice)
r = 999999

assert shared_compare(x, r, alice, bob).value == 0

In [ ]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
x = PrivateScalar(0, alice)
r = 2**l - 1

assert shared_compare(x, r, alice, bob).value == 0

In [ ]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
x = PrivateScalar(2**l - 1, alice)
r = 2**l - 1

assert shared_compare(x, r, alice, bob).value == 0